# Morpheus CLMP (ver. 1.0)

***

Powered by tegridy-tools TMIDIX Optimus Processors: https://github.com/asigalov61/tegridy-tools

***

Credit for GPT2-RGA code used in this colab goes out @ Sashmark97 https://github.com/Sashmark97/midigen and @ Damon Gwinn https://github.com/gwinndr/MusicTransformer-Pytorch

***

WARNING: This complete implementation is a functioning model of the Artificial Intelligence. Please excercise great humility, care, and respect. https://www.nscai.gov/

***

#### Project Los Angeles

#### Tegridy Code 2021

***

# (Setup Environment)

In [ ]:
#@title nvidia-smi gpu check
!nvidia-smi

In [ ]:
#@title Install all dependencies (run only once per session)

!git clone https://github.com/asigalov61/tegridy-tools
!pip install torch
!pip install tqdm

In [ ]:
#@title Import all needed modules

print('Loading needed modules. Please wait...')
import os
import tqdm as tqdm

if not os.path.exists('/notebooks/Dataset'):
    os.makedirs('/notebooks/Dataset')

print('Loading TMIDIX module...')
os.chdir('/notebooks/tegridy-tools/tegridy-tools')
import TMIDIX

os.chdir('/notebooks/tegridy-tools/tegridy-tools')
from GPT2RGAX import *

os.chdir('/notebooks/')

print('Loading complete!!! Enjoy! :)')

# (MODEL)

In [ ]:
# Download CLMP Model
print('Donwloading the model...Please wait...')
!wget --no-check-certificate -O 'Morpheus-CLMP-Trained-Model.pth' "https://onedrive.live.com/download?cid=8A0D502FC99C608F&resid=8A0D502FC99C608F%2118523&authkey=AKWZMkzvZv3WBSo"
print('Donwloaded! Enjoy! :)')

# (LOAD)

In [ ]:
#@title Load/Reload the model
full_path_to_model_checkpoint = "/notebooks/Morpheus-CLMP-Trained-Model.pth" #@param {type:"string"}

print('Loading the model...')
config = GPTConfig(5890, 
                   1024,
                   dim_feedforward=1024,
                   n_layer=6, 
                   n_head=8, 
                   n_embd=1024,
                   enable_rpr=True,
                   er_len=1024)

model = GPT(config).to(get_device())

model.load_state_dict(torch.load(full_path_to_model_checkpoint))

model.eval()
print('Done!')

# (GENERATE MUSIC)

In [ ]:
# Generate CLMP Music

# FIRST SAMPLE MAY NOT BE COHERENT, SO PLEASE GENERATE A FEW SAMPLES FOR EACH PROMPT

LANGUAGE_INPUT_SEQUENCE = 'Can you feel the love tonight' # English letters only. No special or other chars, please.

number_of_instruments = 12 # Change number of instruments if desired (1-12)

number_of_tokens_to_generate = 1024 # Up to 1024 tokens

temperature = 0.8 # Recommended temperature 0.7-0.8

show_stats = False # Set this to True if you need detailed output MIDI stats

#===================================================================

print('=' * 70)
print('Morpheus CLMP Model Generator')
print('=' * 70)

def str2ints(string):
    return [5888] + [ord(y)+(256 * 11)+(256 * 11) for y in string.lower()] + [5888]

sequence = str2ints(LANGUAGE_INPUT_SEQUENCE)

print('Input SEQ:', LANGUAGE_INPUT_SEQUENCE)
print('=' * 70)

print('Input SEQ INTs:', sequence)
print('=' * 70)

print('Generating output...Please wait...')
print('=' * 70)

tokens_range = (256 * 11) + (256 * number_of_instruments)

rand_seq = model.generate(torch.Tensor(sequence), 
                              target_seq_length=number_of_tokens_to_generate, 
                              temperature=temperature,
                              stop_token=tokens_range,
                              verbose=True)

out = rand_seq[0].cpu().numpy().tolist()

print('Generation complete...')
print('=' * 70)

print('Converting to MIDI...')
print('=' * 70)

if len(out) != 0:
    song = []
    song = out
    song_f = []
    time = 0
    dur = 0
    vel = 0
    pitch = 0
    duration = 0
    once = True
    for s in song:
        if s >= 0 and s <= 256 * 11:
            time += s % 256
            dur = ((s // 256) + 1) * 250

        if s >= 256 * 11 and s < (256 * 21):
            if (s // 128) % 2 != 0:
                vel = 80 + (s % 256) % 24
                channel = ((s-128-(256*11)) // 256)
            else:
                vel = 64 + (s % 256) % 24
                channel = ((s-(256*11)) // 256)

            pitch = s % 256

            song_f.append(['note', (abs(time))*10, dur, channel, pitch, vel ])

    detailed_stats = TMIDIX.Tegridy_SONG_to_MIDI_Converter(song_f,
                                                          output_signature = 'Morpheus CLMP',  
                                                          output_file_name = '/notebooks/Morpheus-Music-Composition', 
                                                          track_name='SONG: ' + LANGUAGE_INPUT_SEQUENCE, 
                                                          number_of_ticks_per_quarter=500)

    print('Done!')

    if show_stats:
        print('=' * 70)
        print('Detailed MIDI stats:')
        for key, value in detailed_stats.items():
            print('=' * 70)
            print(key, '|', value)

    print('=' * 70)

else:
    print('Models output is empty! Check the code...')
    print('Shutting down...')
    print('=' * 70)

def ints2string(ints):
    return ''.join([chr(y-(256 * 11)-(256 * 11)) for y in ints if y >= (256 * 11)+(256 * 11) and y < 5888 ])

print('Detected prompt + text:', ints2string(out))

print('=' * 70)

# Congrats! You did it! :)